In [35]:
# Based on - https://github.com/genicam/harvesters

from harvesters.core import Harvester
from PIL import Image
from datetime import datetime 
import time
import copy
class Imager:
    def __init__(self):
        self.h = Harvester()
        self.h.add_file(r'C:\\Program Files\\SVS-VISTEK GmbH\\SVCam Kit\\TLUsb\\bin\\sv_u3v_tl_x64.cti') 
        self.h.add_file(r'C:\Program Files\SVS-VISTEK GmbH\SVCam Kit\TLGigE\bin\\sv_gev_tl_x64.cti') 
        self.h.update()
        num_devices = len(self.h.device_info_list)
        print(f"Found {num_devices} devices") # see all cameras
        if num_devices == 0:
            print("No devices found. If the camera is connected - call imager.clear_all() or restart the kernel and try again.")
        self.serial_ids = [info.serial_number for info in self.h.device_info_list]
        print(f"Devices IDs - {self.serial_ids}") # see all cameras
        self.cams = [self.h.create_image_acquirer(serial_number=_id) for _id in self.serial_ids]
        
    def get_camera_params(self):
        """
        List of possible parameters to adjust"""
        print(dir(self.cams[0].remote_device.node_map)) # see all parameters

    def update_params(self, ExposureTime, PixelFormat, AcquisitionFrameRate):
        for ia in self.cams:
            ia.remote_device.node_map.ExposureTime.value = ExposureTime # for example, change exposure time
            ia.remote_device.node_map.PixelFormat.value = PixelFormat # .symbolics instead of .value for options
            ia.remote_device.node_map.AcquisitionFrameRate.value = AcquisitionFrameRate
    
    def get_temperature(self):
        for ia in self.cams:
            print(ia.remote_device.node_map.DeviceTemperature.value)
            
    def _start_acquisitions(self):
        for ia in self.cams:
            ia.start_acquisition(run_in_background=True) # Start capturing images

    def _stop_acquisitions(self):
        for ia in self.cams:
            ia.stop_acquisition() # stop capturing images
            
    def process_components_to_images(self, components):
        images = []
        for component in components:

            _2d = data.reshape(component.height, component.width)
            print(_2d.min(), _2d.max())
            images.append(Image.fromarray(_2d))
        return images
    
    def show_images(self):
        for img, img_time, cam_id in self.images:
            img.show()
            img.save(f'{cam_id}_{img_time.strftime("%Y_%m_%d_%H%M%S")}.jpeg')
            
    def get_images(self, show_images):
        images_with_times=[]
        for cam_id, ia in zip(self.serial_ids, self.cams):
            # acquire and save an image
            cur_time = datetime.now()

            with ia.fetch_buffer() as buffer: 
                component = buffer.payload.components[0]
                print(f"cam {cam_id} captured image of format {component.data_format} at {cur_time}")
                if component.data_format == 'Mono12Packed':
                    data = component.data >> 4
                else:
                    data = component.data
                _2d = component.data.reshape(component.height, component.width)

                img = Image.fromarray(copy.deepcopy(_2d))
#                 img.show()
#                 img.save(f'{cam_id}_{cur_time.strftime("%Y_%m_%d_%H%M%S")}.jpeg')
                images_with_times.append((img, cur_time, cam_id))

        self.images = images_with_times

    def clear_all(self):
        for ia in self.cams:
            ia.destroy()
        self.h.reset()


imager = Imager()

Found 2 devices
Devices IDs - ['101935', '192900073']


In [36]:
imager._start_acquisitions()

In [37]:
#imager.update_params(ExposureTime=30000, PixelFormat='Mono8', AcquisitionFrameRate = 10)
imager.get_images(show_images=True)
#imager.get_images(show_images=True)

cam 101935 captured image of format Mono8 at 2021-05-30 19:04:06.390677
cam 192900073 captured image of format BayerRG8 at 2021-05-30 19:04:06.394676


In [38]:
imager.show_images()

In [34]:
imager._stop_acquisitions()
imager.clear_all()